# Download all institutions names and ids

Author: Romain THOMAS  
June 2023  
Stockholm Resilience Centre, Stockholm University

In [1]:
# from pyalex import Institutions
from tqdm import tqdm
import pandas as pd

import os, sys
sys.path.append(os.path.abspath('pyalex'))
from pyalex import Institutions


list_of_institutions_file_path = "list_all_institutions.parquet"

per_page = 200

# get the number of entities to download
results, meta = Institutions().get(per_page = 1, return_meta=True)
n_entities = meta['count']

entities_list = [None] * n_entities

pager = Institutions().paginate(per_page = per_page, n_max = None)

with tqdm(total=n_entities) as pbar:
    i = 0
    for page in pager:
        for entitie in page:
            # delete useless datas
            del entitie['international']
            del entitie['repositories']
            del entitie['country_code'] # already in geo.country_code
    
            # keep only the first element in the list of theses data
            # display_name_acronym
            if entitie['display_name_acronyms']:
                entitie['display_name_acronym'] = entitie['display_name_acronyms'][0]
            else:
                entitie['display_name_acronym'] = None
            del entitie['display_name_acronyms']
            # display_name_alternative
            if entitie['display_name_alternatives']:
                entitie['display_name_alternative'] = entitie['display_name_alternatives'][0]
            else:
                entitie['display_name_alternative'] = None
            del entitie['display_name_alternatives']
            # add computed datas:
            # works_cited_by_count_average
            if entitie['works_count'] == 0:
                entitie['works_cited_by_count_average'] = 0
            else:
                entitie['works_cited_by_count_average'] = round(entitie['cited_by_count']/entitie['works_count'], 2)
            
            entities_list[i] = entitie
            i += 1
        # update the progress bar
        pbar.update(per_page)

entities_list_df = pd.json_normalize(entities_list)
entities_list_df.to_parquet(list_of_institutions_file_path, compression=None)

102600it [09:25, 181.47it/s]                                                                                                                                                                                       
